* 1段階目で特徴を抽出
* 2段階目でコード生成

In [3]:
import pandas as pd
import numpy as np
from langchain.llms import OpenAI

import sys
sys.path.append('../..')

from src.data.moon import moon_data
from src.data.pseudodata import pseudo_data
from src.data.titanic import titanic_data

from src.utils.data_to_text import data_to_text

In [2]:
df_train, df_test = moon_data(num_train=300, seed = 3655)
df_train.head()

train data:300
test data:700


,Feature_1,Feature_2,target
644,0.069,-0.037,1
609,0.848,0.507,0
548,0.343,-0.249,1
513,-0.578,0.914,0
994,1.488,-0.376,1


In [6]:
train_str = data_to_text(df_train)

In [ ]:
prompt = """

"""

In [ ]:
llm_model = OpenAI(temperature=0, model_name = 'gpt-4-0613')
llm_model(prompt)